In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
from keras.models import Sequential,Model
from keras.layers import Dense , Activation , Input , LSTM , Flatten , Dropout
from keras.callbacks import EarlyStopping , ModelCheckpoint

# Notes  
CNN flow : 이미지 -> 합성곱 계층 ->  신경망 -> 출력  
  
    
\*합성곱 계층 : {filter-> relu} X N (병렬식)  
- filter 역할 : 이미지에서 특징값을 추출  
- relu 역할 : 특징값을 비선형 값으로 변형하는 활성화 함수  
---
ㄴ

__MLP로__

In [17]:
import keras.datasets as kd
(x_train, y_train), (x_test, y_test) = kd.mnist.load_data()

In [5]:
x_train.shape , x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [10]:
gray_scale = 255
x_train = x_train/gray_scale
x_test = x_test/gray_scale

In [18]:
model = Sequential([
    Flatten(input_shape=(28, 28)), # 데이터 차원 변경
    Dense(256, activation='relu'), # 첫번째 히든 레이어 (h1)
    Dense(128, activation='relu'), # 두번째 히든 레이어 (h2)
    Dropout(0.1), # 두번째 히든 레이어(h2)에 드랍아웃(10%) 적용
    Dense(10), # 세번째 히든 레이어 (logit)
    Activation('softmax') # softmax layer
])


In [19]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [20]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5)]
model.fit(x_train, y_train, epochs=300, batch_size=1000, validation_split = 0.2, callbacks=callbacks)
results = model.evaluate(x_test,  y_test, verbose = 1)

Epoch 1/300
48/48 [==============================] - 1s 17ms/step - loss: 9.1899 - accuracy: 0.7556 - val_loss: 1.2617 - val_accuracy: 0.8687
Epoch 2/300
48/48 [==============================] - 1s 14ms/step - loss: 1.0527 - accuracy: 0.8344 - val_loss: 0.6631 - val_accuracy: 0.8656
Epoch 3/300
48/48 [==============================] - 1s 14ms/step - loss: 0.6265 - accuracy: 0.8635 - val_loss: 0.4938 - val_accuracy: 0.8935
Epoch 4/300
48/48 [==============================] - 1s 14ms/step - loss: 0.4434 - accuracy: 0.8919 - val_loss: 0.4240 - val_accuracy: 0.9131
Epoch 5/300
48/48 [==============================] - 1s 14ms/step - loss: 0.3484 - accuracy: 0.9116 - val_loss: 0.3843 - val_accuracy: 0.9204
Epoch 6/300
48/48 [==============================] - 1s 14ms/step - loss: 0.2869 - accuracy: 0.9237 - val_loss: 0.3704 - val_accuracy: 0.9244
Epoch 7/300
48/48 [==============================] - 1s 14ms/step - loss: 0.2516 - accuracy: 0.9298 - val_loss: 0.3414 - val_accuracy: 0.9320
Epoch 

In [21]:
results

[0.32740500569343567, 0.9603000283241272]

__CNN으로__

In [27]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical

In [23]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.reshape(x_train, (60000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [25]:
gray_scale = 255
x_train = x_train/gray_scale
x_test = x_test/gray_scale

In [28]:
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

필터를 16개 , 32개 로 설정해여 Conv2D를 구성해보자

In [30]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(28,28,1),padding='same'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',padding='same'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))


In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               200832    
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

In [32]:
model.compile(loss=categorical_crossentropy,optimizer=Adam(),
              metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2)]

model.fit(x_train, y_train,
          batch_size=500,
          epochs=5,
          verbose=1,
          validation_split = 0.1, 
          callbacks=callbacks)


Epoch 1/5
108/108 [==============================] - 22s 204ms/step - loss: 0.4935 - accuracy: 0.8639 - val_loss: 0.1302 - val_accuracy: 0.9615
Epoch 2/5
108/108 [==============================] - 22s 200ms/step - loss: 0.1191 - accuracy: 0.9638 - val_loss: 0.0701 - val_accuracy: 0.9802
Epoch 3/5
108/108 [==============================] - 21s 194ms/step - loss: 0.0738 - accuracy: 0.9774 - val_loss: 0.0596 - val_accuracy: 0.9842
Epoch 4/5
108/108 [==============================] - 21s 194ms/step - loss: 0.0585 - accuracy: 0.9818 - val_loss: 0.0560 - val_accuracy: 0.9837
Epoch 5/5
108/108 [==============================] - 21s 194ms/step - loss: 0.0487 - accuracy: 0.9855 - val_loss: 0.0449 - val_accuracy: 0.9870


In [33]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03994368389248848
Test accuracy: 0.9866999983787537
